In [1]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import Model
import random
from tensorflow.keras.utils import to_categorical

config = tf.ConfigProto()  
config.gpu_options.allow_growth=True
session = tf.Session(config=config)
K.set_session(session)

In [2]:
def random_crop(img): 
    width = img.size[0]
    height = img.size[1]
    
    scale = random.uniform(0.85, 1.0)
    px = random.uniform(0.0, 1.0 - scale)*width
    py = random.uniform(0.0, 1.0 - scale)*height
    
    
    img = img.crop((int(px), int(py), int(px+scale*width)-1, int(py+scale*height)-1 )) 
    return img 

def random_flip_left_right(img): 
    prob = random.randint(0,1) 
    if prob == 1: 
        img = img.transpose(Image.FLIP_LEFT_RIGHT) 
    return img



def mk_generator(batchsize, file_path, shuffle = True, train = True):
    with open(file_path, 'r') as f:
        lines = f.readlines()
        if shuffle:
            random.shuffle(lines)
        
        datas = []
        labels = []
        imgPaths = []
        
        i=0
        
        while True:
            line = lines[i%len(lines)]
            
            paras = line.strip('\n').split(' ')
            imagePath = paras[0]
            label = paras[1]
            img = Image.open(imagePath)
            if train:
                img = random_flip_left_right(img)
                img = random_crop(img)

            data = np.array(img.resize((224,224)), dtype = np.float32)/127.5 - 1. #FROM keras file
            
            
            datas.append(data)
            labels.append(label)
            imgPaths.append(line)
            
            i+=1
            
            if (i%batchsize) == 0:
                if train:
                    yield np.array(datas), np.array(to_categorical(labels, 2))
                else:
                    yield np.array(datas), np.array(to_categorical(labels, 2)), imgPaths
                    
                datas = []
                labels = []
                imgPaths = []
                
            if i % len(lines) == 0 and shuffle:
                print 'shuffle'
                random.shuffle(lines)

In [3]:
batch_size = 32
num_samples = 14299

K.set_learning_phase(1)
base_model = MobileNetV2(input_shape=(224, 224, 3), alpha=1.4, include_top=False)

for layer in base_model.layers:
    layer.kernel_regularizer = keras.regularizers.l2(0.05)
    #layer.trainable = False

inputs = base_model.input
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
y = keras.layers.Dense(2, activation='softmax',kernel_regularizer = keras.regularizers.l2(0.05))(x)

model = Model(inputs = inputs, outputs = y)

adam = keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

model.load_weights('./tt_mobilenet.h5', by_name=True);
generator = mk_generator(batchsize=batch_size, file_path = '/home/sw/data/yc/yaochang/2dtrain.txt')

savemodel = keras.callbacks.ModelCheckpoint('mobilenet.h5', verbose=1)
model.fit_generator(generator, num_samples/batch_size, epochs=10, callbacks=[savemodel])



Epoch 1/10
445/446 [============================>.] - ETA: 0s - loss: 0.0344 - acc: 0.9915
Epoch 00001: saving model to mobilenet.h5
446/446 [==============================] - 87s 196ms/step - loss: 0.0343 - acc: 0.9915
Epoch 2/10
445/446 [============================>.] - ETA: 0s - loss: 0.0243 - acc: 0.9944
Epoch 00002: saving model to mobilenet.h5
446/446 [==============================] - 79s 177ms/step - loss: 0.0244 - acc: 0.9943
Epoch 3/10
445/446 [============================>.] - ETA: 0s - loss: 0.0253 - acc: 0.9939
Epoch 00003: saving model to mobilenet.h5
446/446 [==============================] - 80s 178ms/step - loss: 0.0253 - acc: 0.9939
Epoch 4/10
 48/446 [==>...........................] - ETA: 1:11 - loss: 0.0226 - acc: 0.9954

KeyboardInterrupt: 

In [4]:
batch_size =1
num_samples = 34

K.set_learning_phase(0)
base_model = MobileNetV2(input_shape=(224, 224, 3), alpha=1.4, include_top=False)

inputs = base_model.input
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
y = keras.layers.Dense(2, activation='softmax')(x)

model = Model(inputs = inputs, outputs = y)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.load_weights('mobilenet.h5')

generator = mk_generator(batchsize=batch_size, file_path = '/home/sw/data/1101-tag/2dtest.txt', shuffle = False, train = False)

for i in range(num_samples/batch_size):
    img, label, imgnames = next(generator)
    outputs = model.predict(img, batch_size=batch_size)
    res = np.equal(np.argmax(outputs, -1),np.argmax(label,-1)).astype(np.float32)
    print np.array(imgnames)[np.where(res==0)]
    print np.mean(res)

['/home/sw/data/1101-tag/1/1_1541052336_26f01cea-dd9c-11e8-ad10-6045cb7fb916/0004.jpg 1\n']
0.0
['/home/sw/data/1101-tag/1/1_1541052336_26f01cea-dd9c-11e8-ad10-6045cb7fb916/0002.jpg 1\n']
0.0
[]
1.0
[]
1.0
[]
1.0
[]
1.0
[]
1.0
[]
1.0
[]
1.0
[]
1.0
['/home/sw/data/1101-tag/1/1_1541053456_c24da714-dd9e-11e8-ad10-6045cb7fb916/0001.jpg 1\n']
0.0
['/home/sw/data/1101-tag/1/1_1541053456_c24da714-dd9e-11e8-ad10-6045cb7fb916/0014.jpg 1\n']
0.0
['/home/sw/data/1101-tag/1/1_1541053661_3c90d0dc-dd9f-11e8-ad10-6045cb7fb916/0012.jpg 1\n']
0.0
['/home/sw/data/1101-tag/1/1_1541053661_3c90d0dc-dd9f-11e8-ad10-6045cb7fb916/0009.jpg 1\n']
0.0
['/home/sw/data/1101-tag/1/1_1541053821_9bbe7899-dd9f-11e8-ad10-6045cb7fb916/0000.jpg 1\n']
0.0
['/home/sw/data/1101-tag/1/1_1541053821_9bbe7899-dd9f-11e8-ad10-6045cb7fb916/0010.jpg 1\n']
0.0
['/home/sw/data/1101-tag/1/1_1541054074_32a92385-dda0-11e8-ad10-6045cb7fb916/0015.jpg 1\n']
0.0
['/home/sw/data/1101-tag/1/1_1541054074_32a92385-dda0-11e8-ad10-6045cb7fb916/000